In [ ]:
import datasets

path = ""
dataset_before = datasets.load_from_disk(path)

In [ ]:
dataset_before

In [ ]:
dataset_before[1]['text']

In [ ]:
dataset_before[1]['query']

In [ ]:
dataset_before[1]['input_for_gemma']

In [ ]:
stop_token_ids = [
    tokenizer.convert_tokens_to_ids(x) for x in [[tokenizer.eos_token], ["<", "|", "user"]]
]

stop_token_ids = [torch.LongTensor(x).to(device) for x in stop_token_ids]


from transformers import StoppingCriteria, StoppingCriteriaList


class StopOnTokens(StoppingCriteria):
    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        for stop_ids in stop_token_ids:
            if torch.eq(input_ids[0][-len(stop_ids):], stop_ids).all():
                return True
        return False

stopping_criteria = StoppingCriteriaList([StopOnTokens()])

In [ ]:
import requests

API_URL = "https://xxxx.xxxx.aws.endpoints.huggingface.cloud"
headers = {
	"Accept" : "application/json",
	"Authorization": "Bearer xxxx",
	"Content-Type": "application/json" 
}

def query(payload):
	response = requests.post(API_URL, headers=headers, json=payload)
	return response.json()

output = query({
	"inputs": "Can you please let us know more details about your ",
	"parameters": {
        "top_k": 0,
		"top_p": 0.15,
		"temperature": 0.2,
		"max_new_tokens": 512,
        
    }
})

In [ ]:
from tqdm import tqdm

completion = []

# instruction = "You are a professional requirements engineer who helps users brainstorm more software requirements."

for input_for_gemma in tqdm(dataset_before['input_for_gemma']):
    # result = pipe(f"<|system|>\n{instruction}\n<|user|>\n{query}\n<|assistant|>\n")
    result = query({
	"inputs": input_for_gemma,
	"parameters": {}
})
    
    # result = pipe(input_for_gemma)
    result = result[0]['generated_text'].split('\n<|assistant|>\n')[-1]
    # result = result.strip(" ")
    print('```````````````````````````')
    print(result)
    completion.append(result)

In [ ]:
updated_dataset = dataset_before.add_column("gemma_preds", completion)
updated_dataset.save_to_disk("E:\\files\\Stuttgart\\Thesis\\NLP\\saved_datasets\\final_dataset_gemma_09052024\\my_gemma_after_prediction_16052024")

In [ ]:
import json

dataset_dict = updated_dataset.to_dict()

with open('E:\\files\\Stuttgart\\Thesis\\NLP\\saved_datasets\\final_dataset_gemma_09052024\\my_gemma_after_prediction_16052024.json', 'w') as f:
    json.dump(dataset_dict, f, indent=4)

In [ ]:
llama2_datasets = datasets.load_from_disk("E:\\files\\Stuttgart\\Thesis\\NLP\\saved_datasets\\final_dataset_for_training_17042024\\my_llama_after_prediction_01052024")


dataset_dict = llama2_datasets.to_dict()
with open('E:\\files\\Stuttgart\\Thesis\\NLP\\saved_datasets\\final_dataset_gemma_09052024\\my_llama_after_prediction_16052024.json', 'w') as f:
    json.dump(dataset_dict, f, indent=4)

In [ ]:

import json
from datasets import Dataset
with open('E:\\files\\Stuttgart\\Thesis\\NLP\\saved_datasets\\final_dataset_gemma_09052024\\my_gemma_after_prediction_23052024.json', 'r') as f:
    dataset_dict_gemma = json.load(f)

# 将字典数据转换为 datasets.Dataset
dataset = Dataset.from_dict(dataset_dict_gemma)

print(dataset[0]['gemma_preds'])

In [ ]:
for i in range(30):
    print('``````````````````````',i,)
    print(dataset[i]['gemma_preds'])


In [ ]:
dataset.save_to_disk("E:\\files\\Stuttgart\\Thesis\\NLP\\saved_datasets\\final_dataset_gemma_09052024\\my_gemma_after_prediction_23052024")